In [1]:
from reactionmodel.model import Species, Reaction, ReactionRateFamily, Model
import reactionmodel.parser as parser
import yaml

In [2]:
from reactionmodel.model import Species, Reaction, Model
A = Species('A')
r = Reaction([A], [], description='death', k='gamma')
m = Model([A], [r])

m.save('./model.yaml')

In [3]:
m == m.from_dict(m.to_dict())

True

In [4]:
m.to_dict()

{'species': [{'name': 'A'}],
 'reactions': [{'description': 'death',
   'reactants': ['A'],
   'products': [],
   'k': 'gamma'}]}

In [9]:
m.reaction_groups

[Reaction(description=death, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=gamma)]

In [10]:
m.from_dict(m.to_dict()).reaction_groups

[Reaction(description=death, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=None)]

In [3]:
from itertools import product
import numpy as np

As = ['x', 'y', 'z']
Bs = ['p', 'q', 'r']

Q = np.array(range(9)).reshape(3,3)

A_names = [f'A_{a}' for a in As]
B_names = [f'B_{b}' for b in Bs]
a_species   = [Species(a) for a in A_names]
b_species   = [Species(b) for b in B_names]

reactions = [
    Reaction(
        [(a, 2)],
        b,
        description=f'2{a.name} => {b.name}',
        k=f'Q[{A_names.index(a.name)}][{B_names.index(b.name)}]'
    )
    for a,b in product(a_species, b_species)
]
m = Model(a_species+b_species, reactions)
m.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_y', description=''), 2),), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Speci

In [4]:
from yaml import SafeLoader as Loader

yaml_string = """
families:
  As: [x, y, z]
  Bs: [p, q, r]

species:
- name: 'A_$i'
  used_families: {'i': 'As'}
- name: 'B_$j'
  used_families: {'j': 'Bs'}

reactions:
- description: 2A_$i => B_$j
  reactants: [['A_$i',2]]
  products: ['B_$j']
  k: 'Q[#i][#j]'
  used_families: {'i': 'As', 'j': 'Bs'}

parameters:
 foo: 2.0
 Q: {'path': './examples/minimal/q.csv'}

simulator_config:
  simulator: gillespie

initial_condition:
  A_$i: 1.0 * foo
  used_families: {'i': 'As'}
"""

md = yaml.load(yaml_string, Loader=Loader)
md

{'families': {'As': ['x', 'y', 'z'], 'Bs': ['p', 'q', 'r']},
 'species': [{'name': 'A_$i', 'used_families': {'i': 'As'}},
  {'name': 'B_$j', 'used_families': {'j': 'Bs'}}],
 'reactions': [{'description': '2A_$i => B_$j',
   'reactants': [['A_$i', 2]],
   'products': ['B_$j'],
   'k': 'Q[#i][#j]',
   'used_families': {'i': 'As', 'j': 'Bs'}}],
 'parameters': {'foo': 2.0, 'Q': {'path': './examples/minimal/q.csv'}},
 'simulator_config': {'simulator': 'gillespie'},
 'initial_condition': {'A_$i': '1.0 * foo', 'used_families': {'i': 'As'}}}

In [5]:
p = parser.loads(md)

FileNotFoundError: [Errno 2] No such file or directory: './examples/minimal/q.csv'

In [6]:
p.model == m

NameError: name 'p' is not defined

In [6]:
p.model.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_y', description=''), 2),), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Speci

In [7]:
m.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_y', description=''), 2),), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Speci

In [8]:
p.initial_condition

{'A_x': '1.0 * foo', 'A_y': '1.0 * foo', 'A_z': '1.0 * foo'}

In [9]:
p.model.make_initial_condition(p.initial_condition, p.parameters)

Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0
Evaluating expression: 1.0 * foo => 2.0


array([2., 2., 2., 0., 0., 0.])

In [10]:
p.parameters

{'foo': 2.0,
 'Q': array([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])}

In [11]:
p.model.get_k(parameters=p.parameters)

Evaluating expression: Q[0][0] => 1.0
Evaluating expression: Q[0][1] => 2.0
Evaluating expression: Q[0][2] => 3.0
Evaluating expression: Q[1][0] => 4.0
Evaluating expression: Q[1][1] => 5.0
Evaluating expression: Q[1][2] => 6.0
Evaluating expression: Q[2][0] => 7.0
Evaluating expression: Q[2][1] => 8.0
Evaluating expression: Q[2][2] => 9.0


array([1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [12]:
p.simulator_config

{'simulator': 'gillespie'}

In [13]:
p.model.all_reactions

[Reaction(description=2A_x => B_p, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][0]),
 Reaction(description=2A_x => B_q, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][1]),
 Reaction(description=2A_x => B_r, reactants=((Species(name='A_x', description=''), 2),), products=(Species(name='B_r', description=''),), kinetic_order=((Species(name='A_x', description=''), 2),), k=Q[0][2]),
 Reaction(description=2A_y => B_p, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_p', description=''),), kinetic_order=((Species(name='A_y', description=''), 2),), k=Q[1][0]),
 Reaction(description=2A_y => B_q, reactants=((Species(name='A_y', description=''), 2),), products=(Species(name='B_q', description=''),), kinetic_order=((Speci

In [14]:
p

ParseResults(model=<reactionmodel.model.Model object at 0x156c39c70>, parameters={'foo': 2.0, 'Q': array([[1., 2., 3.],
       [4., 5., 6.],
       [7., 8., 9.]])}, initial_condition={'A_x': '1.0 * foo', 'A_y': '1.0 * foo', 'A_z': '1.0 * foo'}, simulator_config={'simulator': 'gillespie'})

In [15]:
a = Species('A')
r1 = Reaction([a], [])
r2 = Reaction([(a, 2)], [])
r_g = ReactionRateFamily([r1, r2], k='hi')

m = Model([a], [r1,r2])
m.to_dict()
from yaml import SafeDumper as Dumper
dump = yaml.dump(m.to_dict(), Dumper=Dumper)

In [16]:
p = parser.loads(yaml.load(dump, Loader=Loader))

{'species': [{'description': '', 'name': 'A'}], 'reactions': [{'description': '', 'k': None, 'kinetic_orders': [{'description': '', 'name': 'A'}], 'products': [], 'reactants': [{'description': '', 'name': 'A'}], 'reversible': False}, {'description': '', 'k': None, 'kinetic_orders': [[{'description': '', 'name': 'A'}, 2]], 'products': [], 'reactants': [[{'description': '', 'name': 'A'}, 2]], 'reversible': False}]}
[Reaction(description=, reactants=(Species(name='A', description=''),), products=(), kinetic_order=(Species(name='A', description=''),), k=None), Reaction(description=, reactants=((Species(name='A', description=''), 2),), products=(), kinetic_order=((Species(name='A', description=''), 2),), k=None)]


In [17]:
p.model == m

True